# Sam Burns Quest Data Analytics
## 23/10/2024

## Part 3: Data Analytics

In [6]:
import boto3
import pandas as pd
import json
from io import BytesIO

# Initialize S3 client
s3_client = boto3.client('s3', aws_access_key_id='****',
                         aws_secret_access_key='****',
                         region_name='eu-north-1')  # client to access my S3 bucket
BUCKET_NAME = 'rearc-quest-sam'

# Define the filenames in your S3 bucket
JSON_FILE_NAME = 'population_data.json'
CSV_FILE_NAME = 'pr.data.0.Current'

# Function to load JSON from S3
def load_json_from_s3():
    try:
        json_obj = s3_client.get_object(Bucket=BUCKET_NAME, Key=JSON_FILE_NAME)
        json_data = json_obj['Body'].read().decode('utf-8')
        json_dict = json.loads(json_data)
        df_json = pd.DataFrame(json_dict)  # Convert JSON data to a DataFrame
        return df_json
    except Exception as e:
        print(f"Error loading JSON: {e}")
        return None

# Function to load CSV from S3
def load_csv_from_s3():
    try:
        csv_obj = s3_client.get_object(Bucket=BUCKET_NAME, Key=CSV_FILE_NAME)
        csv_data = csv_obj['Body']
        df_csv = pd.read_csv(BytesIO(csv_data.read()),delimiter='\t')  # Convert CSV data to a DataFrame
        return df_csv
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return None

# Load the data into DataFrames
df_population = load_json_from_s3()
df_current = load_csv_from_s3()


df_population['Year'] = pd.to_numeric(df_population['Year'])

df_13to18 = df_population[(df_population['Year'] >= 2013) & (df_population['Year'] <= 2018)]
popStdDev = df_13to18['Population'].std()
popMean = df_13to18['Population'].mean()

print(popStdDev, popMean)



df_current.columns = df_current.columns.str.strip()


#Group by Series_ID and Year, summing the values
current_SeriesYear = df_current.groupby(['series_id', 'year']).sum()

# Find the year with the maximum sum of values for each Series ID
id_max = current_SeriesYear.groupby('series_id')['value'].idxmax()

# Extract the corresponding rows
result = current_SeriesYear.loc[id_max].reset_index()
print(result)

# Function to strip whitespace from strings
def strip_whitespace(x):
    return x.strip() if isinstance(x, str) else x

# Apply the function to the entire DataFrame
df_current = df_current.applymap(strip_whitespace)
df_population = df_population.applymap(strip_whitespace)


joinedTable = df_current[['series_id','year','period','value']].merge(df_population[['Year','Population']],
                               right_on='Year',
                               left_on = 'year',
                               how='inner').query("series_id == 'PRS30006032' and period == 'Q01'").drop(columns = "Year")

print(joinedTable)

4257089.5415293295 317437383.0
             series_id  year    value
0    PRS30006011        2022   20.500
1    PRS30006012        2022   17.100
2    PRS30006013        1998  704.125
3    PRS30006021        2010   17.600
4    PRS30006022        2010   12.500
..                 ...   ...      ...
277  PRS88003192        2002  282.800
278  PRS88003193        2023  767.007
279  PRS88003201        2022   36.100
280  PRS88003202        2022   28.900
281  PRS88003203        2023  582.391

[282 rows x 3 columns]
         series_id  year period  value  Population
35     PRS30006032  2013    Q01    0.8   311536594
1265   PRS30006032  2014    Q01   -0.1   314107084
2495   PRS30006032  2015    Q01   -1.6   316515021
3725   PRS30006032  2016    Q01   -1.4   318558162
4955   PRS30006032  2017    Q01    0.7   321004407
6185   PRS30006032  2018    Q01    0.4   322903030
7415   PRS30006032  2019    Q01   -1.6   324697795
8645   PRS30006032  2020    Q01   -6.7   326569308
9875   PRS30006032  2021    Q0